In [1]:
import json

from models.encoder import Encoder

import utils

import argparse,random,logging,numpy,os

import torch

import torch.nn as nn

import torch.nn.functional as F

import numpy as np

from torch.autograd import Variable

from torch.utils.data import DataLoader

from torch.nn.utils import clip_grad_norm

from time import time

from tqdm import tqdm

from util import load_dataset, make_iter, Params
import pandas as pd

In [2]:
with open("./data/train_ex.json") as f:
    examples = [json.loads(line) for line in f]

In [108]:
train_df = pd.DataFrame(columns=["doc", "sent"])

count = 0

for n, ex in enumerate(examples[:100]):
    sents = ex['doc'].split('</s> ')
    sents = [x[3:].replace("</s>", "") for x in sents]
    inputs = ex['input'][3:-4]

    tmp = list()
    tmp.append(ex['doc'])
    tmp.append(sents)

    train_df.loc[count] = tmp
    count += 1

In [5]:
params = Params('config/params.json')

embed = torch.Tensor(np.load("./data/embedding.npz")['embedding'])
with open("./data/word2id.json") as f:
    word2id = json.load(f)
vocab = utils.Vocab(embed, word2id)

In [6]:
"""
ae
"""
# load
pretrained_dict = torch.load("./entire_ae.pt")
model = Encoder(params)
model_dict = model.state_dict()
# 1. filter out unnecessary keys
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
# 2. overwrite entries in the existing state dict
model_dict.update(pretrained_dict)
# 3. load the new state dict
model.load_state_dict(model_dict)

model.to(params.device)
model.eval()

Encoder(
  (token_embedding): Embedding(153824, 256, padding_idx=1)
  (pos_embedding): Embedding(1025, 256)
  (encoder_layers): ModuleList(
    (0): EncoderLayer(
      (layer_norm): LayerNorm((256,), eps=1e-06, elementwise_affine=True)
      (self_attention): MultiHeadAttention(
        (attentions): ModuleList(
          (0): SelfAttention(
            (q_w): Linear(in_features=256, out_features=32, bias=False)
            (k_w): Linear(in_features=256, out_features=32, bias=False)
            (v_w): Linear(in_features=256, out_features=32, bias=False)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): SelfAttention(
            (q_w): Linear(in_features=256, out_features=32, bias=False)
            (k_w): Linear(in_features=256, out_features=32, bias=False)
            (v_w): Linear(in_features=256, out_features=32, bias=False)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (2): SelfAttention(
            (q_w): Linear(in_feat

In [7]:
"""
ae embedding 만들기
"""
doc_num = len(examples)
time_cost = 0
file_id = 1

sent_trunc=100
word_trunc=100
PAD_TOKEN = '<pad>'
PAD_IDX = 0
UNK_TOKEN = '<unk>'
UNK_IDX = 1

def w2i(w):
    if w in word2id:
        return word2id[w]
    else:
        return UNK_IDX

In [126]:
whole_output = list()
sent_output = list()


for sents in train_df['sent']:
    """
    doc
    """
    # trunc or pad sent
    max_sent_len = 0
    batch_sents = []
    for sent in sents:
        words = sent.split()
        if len(words) > word_trunc:
            words = words[:word_trunc]
        batch_sents.append(words)

    features = []
    for sent in batch_sents:
        feature = [w2i(w) for w in sent] + [PAD_IDX for _ in range(word_trunc-len(sent))]
        feature = np.array(feature)
        features.append(feature)

    sent_embedding = features
    
    """
    input

    # trunc or pad sent
    words = inputs.split()
    if len(words) > word_trunc:
        words = words[:word_trunc]
    
    input_feature = [[w2i(w) for w in words] + [PAD_IDX for _ in range(word_trunc-len(words))]]
    """

    features = torch.LongTensor(features)
    features = Variable(features)
    
    
    encoder_output = model(features)

    
    tmp = list()

    for output in encoder_output:
        #result = np.array(output.mean(dim=1).tolist())
        result = np.array(output.tolist())
        
        result = np.nan_to_num(result)
        #print(output.unsqueeze(0).shape)

        tmp.append(result)

    #result = encoder_output.mean(dim=0)
    #result = np.array(result.mean(dim=1).tolist())
    result = np.array(result.tolist())

    whole_output.append(result)
    sent_output.append(tmp) # input_output

In [103]:
whole_output[0].shape

(100,)

In [104]:
sent_output[0][0].shape

(100,)

In [59]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

In [112]:
cnt = 0
for n, sent in enumerate(sent_output):
    tmp = list()
    labels = examples[n]['labels'].split("\n")
    answer = labels.index("1")
    print(answer)
    for i in sent:
        tmp.append(cos_sim(i, whole_output[n]))
    maxcosine = tmp.index(min(tmp))
    print(tmp)
    if answer == maxcosine:
        cnt += 1
    print("max: ", tmp.index(max(tmp)))
    print("min: ", tmp.index(min(tmp)))
    print("="*30)

2
[-0.13549741593579137, -0.009750838162741035, 0.04925700657928063, -0.26253114964476165]
max:  2
min:  3
2
[-0.015284969047925917, -0.14575086610476015, -0.06213124984996928, 0.18876781741311366]
max:  3
min:  1
2
[-0.016915224187911113, -0.022714634683048664, 0.14550112689872322, -0.06955476481428373]
max:  2
min:  3
0
[0.04585616963316472, -0.3681124990644639, -0.018887715909939495, 0.16132834721150566]
max:  3
min:  1
2
[-0.0374459648011547, 0.08058759981631222, 0.0072383331623841266, 0.07032142473212843]
max:  1
min:  0
2
[-0.042743894687769124, 0.10558521616982305, 0.0893217614561004, 0.09484251921188912]
max:  1
min:  0
3
[-0.2830672731523369, -0.3249746014637473, 0.0058415149745423955, -0.058717180829726344]
max:  2
min:  1
2
[-0.06355051244712077, 0.05156973077535639, -0.10298011231653405, 0.030725097158494627]
max:  1
min:  2
1
[-0.05040129486838699, 0.1431994773874329, 0.07290068297989238, 0.010201196454594736]
max:  1
min:  0
1
[0.05116021358996935, -0.07783514669220534, -

In [113]:
cnt

17

In [61]:
import numpy as np
def dist(x,y):   
    return np.sqrt(np.sum((x-y)**2))

In [125]:
cnt = 0
for n, sent in enumerate(sent_output):
    tmp = list()
    labels = examples[n]['labels'].split("\n")
    answer = labels.index("1")
    print(answer)
    for i in sent:
        tmp.append(dist(i, whole_output[n]))
    maxcosine = tmp.index(max(tmp))
    print(tmp)
    if answer == maxcosine:
        cnt += 1
    print("max: ", tmp.index(max(tmp)))
    print("min: ", tmp.index(min(tmp)))
    print("="*30)

2


ValueError: operands could not be broadcast together with shapes (100,) (100,256) 

In [123]:
cnt

21

In [87]:
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
output = cos(whole_output[0], sent_output[0][0])

In [88]:
output

tensor([[ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
        [ 0.8218, -0.6166,  0.9100,  ..., -0.2476,  0.6131,  0.0065],
        [ 0.8420,  0.4001, -0.4663,  ..., -0.1339,  0.5475,  0.0224],
        [ 0.2929,  0.6657,  0.7336,  ..., -0.0712,  0.8906, -0.0017]],
       grad_fn=<DivBackward0>)